<a href="https://colab.research.google.com/github/baker371/k8-data-visualization/blob/lwasampijja-baker/upwork-devs/lwasampijja-baker/Visualise_Zenhub_Pipelines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install flatjson

In [7]:
import requests
import pandas as pd 
import json
import os
from pandas.io.json import json_normalize #package for flattening json in pandas df
import flatjson

import warnings
warnings.filterwarnings('ignore')


In [8]:
def git_connect():
  jtoken                = os.getenv('GITHUB_TOKEN', '')
  url                   = f"https://api.github.com/repos/filetrust/k8-data-visualization/issues"
  headers               = {'Authorization': f'token {jtoken}'}
  r                     = requests.get(url, headers=headers, params={'state': 'all'}).json()
  data                  = json_normalize(r, max_level=1)
  global                  dfn
  dfn                   = pd.DataFrame.from_dict(data)
  dfn                   = dfn[['number', 'assignee.login', 'state']]
  return dfn



In [10]:
def zen_connect():
  ztoken                = ''
  headers               = {'X-Authentication-Token': ztoken, }
  url                   = 'https://api.zenhub.io/p1/repositories/289667349/board'
  r                     = requests.get(url, headers=headers).json()
  data                  = flatjson.dumps(r)
  data1                 = json_normalize(data)
  global                  df 
  df                    = pd.DataFrame.from_dict(data1)
  df                    = df.loc[:, ~df.columns.str.endswith('id')]
  df                    = df.loc[:, ~df.columns.str.endswith('is_epic')]
  df                    = df.loc[:, ~df.columns.str.endswith('position')]
  return df

In [12]:
def new_col():
  global                  new
  new                   = df[df.columns[pd.Series(df.columns).str.startswith('pipelines[0]')]]
  new                   = new.transpose() 
  new.columns           = new.iloc[0]
  new                   = new[1:]
  new                   = new.rename({'New Issues':'number'}, axis=1)
  new['New Issues']     = 'New Issues'
  return new

In [14]:
def backlog():
  global                  bak
  bak                   = df[df.columns[pd.Series(df.columns).str.startswith('pipelines[1]')]]
  bak                   = bak.transpose() 
  bak.columns           = bak.iloc[0]
  bak                   = bak[1:]
  bak                   = bak.rename({'Backlog':'number'}, axis=1)
  bak['Backlog']        = 'Backlog'
  return bak

In [16]:
def progress():
  global                  prog
  prog                  = df[df.columns[pd.Series(df.columns).str.startswith('pipelines[2]')]]
  prog                  = prog.transpose() 
  prog.columns          = prog.iloc[0]
  prog                  = prog[1:]
  prog                  = prog.rename({'In Progress':'number'}, axis=1)
  prog['In Progress']   = 'In Progress'
  return prog

In [18]:
def peer():
  global                  peer
  peer                  = df[df.columns[pd.Series(df.columns).str.startswith('pipelines[3]')]]
  peer                  = peer.transpose() 
  peer.columns          = peer.iloc[0]
  peer                  = peer[1:]
  peer                  = peer.rename({'Peer Review':'number'}, axis=1)
  peer['Peer Review']   = 'Peer Review'
  return peer

In [20]:
def merge():
  dfv                   = pd.concat([new, bak, prog, peer])
  global                  df_merge
  df_merge              = pd.merge(dfn, dfv, on='number')
  df_merge              = df_merge.rename({'assignee.login':'Assigned'}, axis=1)
  df_merge              = df_merge.rename({'state':'Status'}, axis=1)
  df_merge              = df_merge.rename({'number':'Total'}, axis=1)
  return df_merge

In [22]:
def status_table():
  return df_merge.groupby(['Assigned', 'Status']).count()

In [23]:
status_table()

,,Total,New Issues,Backlog,In Progress,Peer Review
Assigned,Status,,,,,
GiuseMSD,open,1,0,0,0,1
amanthegeek,open,1,0,0,1,0
baker371,open,1,0,0,1,0
chrisdiao,open,2,0,0,0,2
lucia15,open,5,0,0,1,4
mnjenga2,open,2,0,2,0,0
